# Fine-tuning transfer Learning

the pre-trained model weights from another model are unfrozen and tweaked during to better suit your own data\

For feature extraction transfer learning, you may only train the top 1-3 layers of a pre-trained model with your own data, in fine-tuning transfer learning, you might train 1-3+ layers of a pre-trained model(where the "+" indicates that many or all of the layers could be trained).

In [2]:
# Are we using a GPU? (if not & you're using Google Colab, go to Runtime -> Change Runtime Type -> Harware Accelerator: GPU )
!nvidia-smi

Sun May 15 18:30:30 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 462.13       Driver Version: 462.13       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 307... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   59C    P8    16W /  N/A |   1386MiB /  8192MiB |      9%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
# Get data (10% of labels)
import zipfile

# Unzip the downloaded file
zip_ref = zipfile.ZipFile("10_food_classes_10_percent.zip", "r")
zip_ref.extractall()
zip_ref.close()

In [4]:
# Create training and test directories
train_dir = "10_food_classes_10_percent/train/"
test_dir = "10_food_classes_10_percent/test/"

In [5]:
# Create data inputs
import tensorflow as tf
IMG_SIZE = (224, 224) # define image size
train_data_10_percent = tf.keras.preprocessing.image_dataset_from_directory(directory=train_dir,
                                                                            image_size=IMG_SIZE,
                                                                            label_mode="categorical", # what type are the labels?
                                                                            batch_size=32) # batch_size is 32 by default, this is generally a good number
test_data_10_percent = tf.keras.preprocessing.image_dataset_from_directory(directory=test_dir,
                                                                           image_size=IMG_SIZE,
                                                                           label_mode="categorical")

Found 750 files belonging to 10 classes.
Found 2500 files belonging to 10 classes.


In [6]:
# Check the training data datatype
train_data_10_percent

<BatchDataset shapes: ((None, 224, 224, 3), (None, 10)), types: (tf.float32, tf.float32)>

https://github.com/mrdbourke/tensorflow-deep-learning/blob/main/05_transfer_learning_in_tensorflow_part_2_fine_tuning.ipynb

In [7]:
# Check out the class names of our dataset
train_data_10_percent.class_names

['chicken_curry',
 'chicken_wings',
 'fried_rice',
 'grilled_salmon',
 'hamburger',
 'ice_cream',
 'pizza',
 'ramen',
 'steak',
 'sushi']

In [8]:
# see an example batch of data
for images, labels in train_data_10_percent.take(1):
    print(images, labels)

tf.Tensor(
[[[[193.11224    80.69898   126.0102   ]
   [194.7602     84.94898   129.02551  ]
   [189.63776    84.35714   124.86224  ]
   ...
   [181.38776    78.74496   116.94899  ]
   [182.26534    80.31636   117.29084  ]
   [180.7498     81.821236  115.923256 ]]

  [[188.90816    82.71939   123.168365 ]
   [191.84694    85.79082   125.77551  ]
   [193.         86.78571   124.85714  ]
   ...
   [183.6889     79.21951   119.132774 ]
   [180.70918    81.20412   118.994896 ]
   [182.35704    87.97442   121.99482  ]]

  [[186.42346    84.78061   117.85204  ]
   [192.38776    86.53062   122.88776  ]
   [195.95409    87.40306   124.21429  ]
   ...
   [174.37766    87.52551   126.1429   ]
   [177.82654    86.57144   124.42856  ]
   [174.87227    84.657974  120.00997  ]]

  ...

  [[206.92346   229.92346   243.92346  ]
   [209.92857   231.92857   245.92857  ]
   [211.        233.        247.       ]
   ...
   [113.806274  126.23486   126.44916  ]
   [140.38284   152.81143   152.81143  ]
   [1

### Model 0: Building a transfer learning model using the Keras Functional API

In [9]:
# 1. Create base model with tf.keras.applications
base_model = tf.keras.applications.EfficientNetB0(include_top=False)

# 2. Freeze the base model (so the pre-learned patterns remain)
base_model.trainable = False

# 3. Create inputs into the base model
inputs = tf.keras.layers.Input(shape=(224, 224, 3), name="input_layer")

# 4. Pass the inputs to the base_model
x = base_model(inputs)
# check data shape after passing it to base_model
print(f"Shape after base_model: {x.shape}")

# 6. Average pool the outputs of the base model (aggregate all the most important information, reduce number of computations)
x = tf.keras.layers.GlobalAveragePooling2D(name="global_average_pooling_layer")(x)
print(f"After GlobalAveragePooling2D(): {x.shape}")

# 7. Create the output activation layer
outputs = tf.keras.layers.Dense(10, activation="softmax", name="output_layer")(x)

# 8. Combine the inputs with the outputs into a model
model_0 = tf.keras.Model(inputs, outputs)

# 9. Compile the model
model_0.compile(loss='categorical_crossentropy',
optimizer=tf.keras.optimizers.Adam(),
metrics=['accuracy'])

# 10. Fit the model (we use less steps for validation so it's faster)
history_10_percent = model_0.fit(train_data_10_percent,
epochs=5,
steps_per_epoch=len(train_data_10_percent),
validation_data=test_data_10_percent,
# Go through less of the validation data so epochs are faster (we want faster experiments!)
validation_steps=int(0.25 * len(test_data_10_percent)))



16719872/16705208 [==============================] - 2s 0us/step
Shape after base_model: (None, 7, 7, 1280)
After GlobalAveragePooling2D(): (None, 1280)
Epoch 1/5
24/24 [==============================] - 15s 200ms/step - loss: 1.9126 - accuracy: 0.4093 - val_loss: 1.3469 - val_accuracy: 0.7220
Epoch 2/5
24/24 [==============================] - 3s 112ms/step - loss: 1.1513 - accuracy: 0.7440 - val_loss: 0.9431 - val_accuracy: 0.7878
Epoch 3/5
24/24 [==============================] - 3s 106ms/step - loss: 0.8521 - accuracy: 0.7987 - val_loss: 0.7580 - val_accuracy: 0.8109
Epoch 4/5
24/24 [==============================] - 3s 104ms/step - loss: 0.6973 - accuracy: 0.8400 - val_loss: 0.6788 - val_accuracy: 0.8322
Epoch 5/5
24/24 [==============================] - 3s 108ms/step - loss: 0.5961 - accuracy: 0.8587 - val_loss: 0.6363 - val_accuracy: 0.8224


Now this is called feature extraction transfer learning, similar to what we did with the TensorFlow Hub models.

In other words, we passed our custom data to a pre-trained model, and then put our own output layer on top to make sure the outputs were tailored to our desired number of classes.

In [10]:
# check layers in our base model
for layer_number, layer in enumerate(base_model.layers):
    print(layer_number, layer.name)

0 input_1
1 rescaling
2 normalization
3 stem_conv_pad
4 stem_conv
5 stem_bn
6 stem_activation
7 block1a_dwconv
8 block1a_bn
9 block1a_activation
10 block1a_se_squeeze
11 block1a_se_reshape
12 block1a_se_reduce
13 block1a_se_expand
14 block1a_se_excite
15 block1a_project_conv
16 block1a_project_bn
17 block2a_expand_conv
18 block2a_expand_bn
19 block2a_expand_activation
20 block2a_dwconv_pad
21 block2a_dwconv
22 block2a_bn
23 block2a_activation
24 block2a_se_squeeze
25 block2a_se_reshape
26 block2a_se_reduce
27 block2a_se_expand
28 block2a_se_excite
29 block2a_project_conv
30 block2a_project_bn
31 block2b_expand_conv
32 block2b_expand_bn
33 block2b_expand_activation
34 block2b_dwconv
35 block2b_bn
36 block2b_activation
37 block2b_se_squeeze
38 block2b_se_reshape
39 block2b_se_reduce
40 block2b_se_expand
41 block2b_se_excite
42 block2b_project_conv
43 block2b_project_bn
44 block2b_drop
45 block2b_add
46 block3a_expand_conv
47 block3a_expand_bn
48 block3a_expand_activation
49 block3a_dwcon